In [ ]:
%load_ext tensorboard
%tensorboard --logdir .

In [ ]:
# training command
!nohup yolo task=detect mode=train model=pretrained_models/yolov8x.pt data=data.yaml epochs=100 imgsz="(720,2560)" save_period=1 batch=2 pretrained=pretrained_models/yolov8x.pt plots=true > logs/train/100e.txt 2>&1 &
!nohup tensorboard --logdir runs/detect --port 4000 --bind_all > logs/tensorboard.logs 2>&1 &

In [ ]:
# inference command
!yolo task=detect mode=predict model=runs/detect/train-100e/weights/best.pt source=input_videos/2.mp4 save=true save_txt=true imgsz="(720,2560)" classes=32  2>&1 | tee logs/inference/100e-model.txt

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train-100e/weights/best.pt source=input_videos/2.mp4 save=true save_txt=true imgsz="(720,2560)" classes=32  2>&1 | tee logs/inference/100e-model.txt

In [2]:
!cat logs/inference/100e-model.txt | grep sports | wc -l
!cat logs/inference/original-model-2mp4.txt | grep sports | wc -l

5622
5677


In [12]:
%cd input_videos
# !gdown 1YVpeLLWdpOjPkCRdVTovXOsg2soVOF7O  # primary view
!gdown 17H6gMG99cvqwLBNZWormM4rVS2z2b8HD # secondary view
%cd ..

/workspace/codebase/input_videos


/workspace/env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Downloading...
From: https://drive.google.com/uc?id=17H6gMG99cvqwLBNZWormM4rVS2z2b8HD
To: /workspace/codebase/input_videos/pi1-20240329_215144.mp4
100%|████████████████████████████████████████| 105M/105M [00:02<00:00, 50.8MB/s]
/workspace/codebase


In [13]:
# getting clip from video

# input_video_path = "./input_videos/pi3-20240329_215144.mp4" # primary view
input_video_path = "input_videos/pi1-20240329_215144.mp4" # secondary view

starting_index = 1150
ending_index = 1250
video_ext = input_video_path.split(".")[-1]
output_video_path = input_video_path.replace(f'.{video_ext}', f'_{starting_index}_{ending_index}.{video_ext}')

import cv2
vid = cv2.VideoCapture(input_video_path)
vid_writer = cv2.VideoWriter(output_video_path,
    cv2.VideoWriter_fourcc(*"mp4v"),
    int(vid.get(cv2.CAP_PROP_FPS)),
    (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
)
vid.set(cv2.CAP_PROP_POS_FRAMES, starting_index)
for i in range(starting_index,ending_index):
    s, f = vid.read()
    if not s:
        raise Exception("Can't read frame from video at %d " % i)
    vid_writer.write(f)
vid_writer.release()
vid.release()

In [ ]:
!yolo task=detect mode=track model=pretrained_models/yolov8x.pt source=input_videos/pi3-20240329_215144_1150_1250.mp4 save=true save_txt=true save_conf=true conf='0.01' imgsz="(1080,1920)" classes=32 tracker="botsort.yaml"  2>&1 | tee logs/track/original_model_edge_case_detect_botsort.txt

In [9]:
index=1197
import cv2
input_video_path="input_videos/pi3-20240329_215144.mp4"
vid = cv2.VideoCapture(input_video_path)

vid.set(cv2.CAP_PROP_POS_FRAMES, index)
s, f = vid.read()
cv2.imwrite(f"{index}.png", f)

True

In [12]:
!yolo task=detect mode=predict model=pretrained_models/yolov8x.pt source="1197_ball.png" save=true save_txt=true save_conf=true conf='0.01' classes=32  2>&1

Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

image 1/1 /workspace/codebase/1197_ball.png: 576x640 2 sports balls, 88.3ms
Speed: 4.8ms preprocess, 88.3ms inference, 3886.2ms postprocess per image at shape (1, 3, 576, 640)
Results saved to runs/detect/predict3
1 label saved to runs/detect/predict3/labels
💡 Learn more at https://docs.ultralytics.com/modes/predict
